In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np
import json

# Wczytanie danych z pliku JSON
file_path = 'mnt/data/data (6).json'  # Upewnij się, że ścieżka jest poprawna
with open(file_path, 'r') as f:
    data = json.load(f)

# Rozpakowanie danych z JSON
extracted_data = []
for timestamp, entry in data.items():
    if isinstance(entry, dict) and 'meals' in entry:
        glucose = entry.get('glucose', None)
        insulin_per_ww = entry.get('insulin_per_ww', None)
        insulin_dose = entry.get('insulin_dose', None)
        carbs = sum(meal.get('carbs', 0) for meal in entry['meals'])
        extracted_data.append({
            'glucose': glucose,
            'carbs': carbs,
            'insulin_per_ww': insulin_per_ww,
            'insulin_dose': insulin_dose
        })

# Tworzenie DataFrame
df = pd.DataFrame(extracted_data).dropna()  # Usuwanie pustych wartości

# Sprawdzenie struktury danych
print(df.head())
df.head()

Empty DataFrame
Columns: []
Index: []


""


In [ ]:
# Założenie: Kolumny w pliku odpowiadają wymaganym danym
X = df[['glucose', 'carbs', 'insulin_per_ww']]
y = df['insulin_dose']

# Podział na zestawy treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tworzenie modelu XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)

# Trenowanie modelu
model.fit(X_train, y_train)


In [ ]:
# Predykcja na zbiorze testowym
y_pred = model.predict(X_test)

# Ocena modelu
mae = mean_absolute_error(y_test, y_pred)
print(f'"Mean Absolute Error: {mae:.2f}"')

In [ ]:
# **TESTOWANIE MODEL:** Użytkownik może ręcznie wpisać dane
def manual_test():
    glucose = float(input("Podaj aktualny poziom cukru: "))
    carbs = float(input("Podaj ilość spożywanych węglowodanów: "))
    insulin_per_ww = float(input("Podaj dawkę insuliny na 1 WW: "))
    
    input_data = np.array([[glucose, carbs, insulin_per_ww]])
    prediction = model.predict(input_data)[0]
    print(f'Przewidywana dawka insuliny: {prediction:.2f} jednostek')

# Uruchomienie testu manualnego
manual_test()